In [21]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Import additional library to properply play videos on jupyter notebook
from IPython.display import clear_output


In [22]:
capR = cv2.VideoCapture('robotR.avi')
capL = cv2.VideoCapture('robotL.avi')
print(capR,capL)

<VideoCapture 00CE6120> <VideoCapture 05EE79F0>


In [24]:

print(capR.isOpened(),capL.isOpened())
while (capR.isOpened() and capL.isOpened()):
    retL, frameL = capL.read()
    #print(frameL.shape)
    cv2.imshow('leftframe',frameL)
    cv2.waitKey(100)
    retR, frameR = capR.read()
    #imgR = cv2.cvtColor(frameR,cv2.COLOR_BGR2GRAY)
    imgR=cv2.cvtColor(frameL[frameR.shape[0]//2-50:frameR.shape[0]//2+50,frameR.shape[1]//2-50:frameR.shape[1]//2+50], cv2.COLOR_BGR2GRAY)
    #imgL = cv2.cvtColor(frameL,cv2.COLOR_BGR2GRAY)
    imgL=cv2.cvtColor(frameL[frameL.shape[0]//2-50:frameL.shape[0]//2+50,frameL.shape[1]//2-50:frameL.shape[1]//2+50], cv2.COLOR_BGR2GRAY)
  
    
    #cv2.imshow('right_image',imgR)
    #cv2.waitKey(1000)
    sift= cv2.xfeatures2d.SIFT_create()
    kpL, desL = sift.detectAndCompute(imgL, None)
    kpR, desR = sift.detectAndCompute(imgR, None)
    imgSift = cv2.drawKeypoints(
        imgL, kpL, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    #cv2.imshow("SIFT Keypoints", imgSift)
    #cv2.waitKey(100)
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)   # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(desL, desR, k=2)
    matchesMask = [[0, 0] for i in range(len(matches))]
    
    good = []
    ptsL = []
    ptsR = []
    for i, (m, n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            matchesMask[i] = [1, 0]
            good.append(m)
            ptsR.append(kpR[m.trainIdx].pt)
            ptsL.append(kpL[m.queryIdx].pt)
            
    draw_params = dict(matchColor=(0, 255, 0),
                       singlePointColor=(255, 0, 0),
                       matchesMask=matchesMask[300:500],
                       flags=cv2.DrawMatchesFlags_DEFAULT)
    keypoint_matches = cv2.drawMatchesKnn(
        imgL, kpL, imgR, kpR, matches[300:500], None, **draw_params)
    #cv2.imshow("Keypoint matches", keypoint_matches)
    #cv2.waitKey(5000)
    
    ptsL = np.int32(ptsL)
    ptsR = np.int32(ptsR)
    fundamental_matrix, inliers = cv2.findFundamentalMat(ptsL, ptsR, cv2.FM_RANSAC)
    # We select only inlier points
    ptsL = ptsL[inliers.ravel() == 1]
    ptsR = ptsR[inliers.ravel() == 1]
    
    def drawlines(imgLsrc, imgRsrc, lines, ptsLsrc, ptsRsrc):
        r, c = imgLsrc.shape
        imgLcolor = cv2.cvtColor(imgLsrc, cv2.COLOR_GRAY2BGR)
        imgRcolor = cv2.cvtColor(imgRsrc, cv2.COLOR_GRAY2BGR)
        
        np.random.seed(1)
        for r, ptL, ptR in zip(lines, ptsLsrc, ptsRsrc):
            color = tuple(np.random.randint(0, 255, 3).tolist())
            x0, y0 = map(int, [0, -r[2]/r[1]])
            x1, y1 = map(int, [c, -(r[2]+r[0]*c)/r[1]])
            img1color = cv2.line(imgLcolor, (x0, y0), (x1, y1), color, 1)
            img1color = cv2.circle(imgLcolor, tuple(ptL), 5, color, -1)
            img2color = cv2.circle(imgRcolor, tuple(ptR), 5, color, -1)
        return imgLcolor, imgRcolor
    
    
    linesL = cv2.computeCorrespondEpilines(
        ptsR.reshape(-1, 1, 2), 2, fundamental_matrix)
    linesL = linesL.reshape(-1, 3)
    img5, img6 = drawlines(imgL, imgR, linesL, ptsL, ptsR)
    linesR = cv2.computeCorrespondEpilines(
        ptsL.reshape(-1, 1, 2), 1, fundamental_matrix)
    
    linesR = linesR.reshape(-1, 3)
    
    img3, img4 = drawlines(imgR, imgL, linesR, ptsR, ptsL)
    
    #plt.subplot(121), plt.imshow(img5)
    #plt.subplot(122), plt.imshow(img3)
    #plt.suptitle("Epilines in both images")
    #plt.show()
    
    # Stereo rectification (uncalibrated variant)
    print(imgL.shape)
    hL, wL = imgL.shape
    hR, wR = imgR.shape
    _, HL, HR = cv2.stereoRectifyUncalibrated(
        np.float32(ptsL), np.float32(ptsR), fundamental_matrix, imgSize=(wL, hL)
    )
    
    imgL_rectified = cv2.warpPerspective(imgL, HL, (wL, hL))
    imgR_rectified = cv2.warpPerspective(imgR, HR, (wR, hR))
    #cv2.imwrite("rectified_left.png", imgL_rectified)
    #cv2.imwrite("rectified_right.png", imgR_rectified)



    
    stereo = cv2.StereoBM_create(numDisparities=64, blockSize=21)
    disparity = stereo.compute(imgR_rectified,imgL_rectified)
    
    cv2.imshow('leftframe_segmented',imgL)
    cv2.waitKey(100)
    

True True
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)
(100, 100)


KeyboardInterrupt: 